In [1]:
!pip install numpy protobuf==3.16.0
!pip install onnx
!pip install -q simpletransformers
!pip install -q datasets transformers[sentencepiece] simpletransformers
!pip install onnxruntime
!pip install transformers[onnx]

     |████████████████████████████████| 1.0 MB 5.2 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


     |████████████████████████████████| 12.8 MB 5.6 MB/s 
     |████████████████████████████████| 248 kB 5.1 MB/s 
     |████████████████████████████████| 9.7 MB 79.1 MB/s 
     |████████████████████████████████| 1.2 MB 72.8 MB/s 
     |████████████████████████████████| 1.7 MB 72.6 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 311 kB 78.9 MB/s 
     |████████████████████████████████| 6.8 MB 69.8 MB/s 
     |████████████████████████████████| 3.5 MB 76.7 MB/s 
     |████████████████████████████████| 596 kB 72.5 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 80.4 MB/s 
     |████████████████████████████████| 144 kB 69.1 MB/s 
     |████████████████████████████████| 181 kB 82.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 134 kB 84.2 MB/s 
     |████████████████████████████████| 243 kB 82.4 MB/s 
     |███████████████

In [1]:
to_predict = ['ไ่ก่ จิก เด็ก ตาย คน _ เกิด บน ปาก โอ่ง']

In [2]:
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]

In [3]:
from transformers.models.camembert import CamembertConfig, CamembertOnnxConfig , CamembertForTokenClassification ,CamembertTokenizer
from onnxruntime import InferenceSession, SessionOptions
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import os
from tqdm.notebook import tqdm
import numpy as np

In [4]:
use_cuda = True
cuda_device=-1

In [5]:
if use_cuda:
    if torch.cuda.is_available():
        if cuda_device == -1:
            device = torch.device("cuda")
        else:
            device = torch.device(f"cuda:{cuda_device}")
    else:
        raise ValueError(
            "'use_cuda' set to True when cuda is unavailable."
            "Make sure CUDA is available or set use_cuda=False."
        )
else:
    device = "cpu"

In [6]:
# จัดการ Onnx runtime

onnx_execution_provider = (["CUDAExecutionProvider"] if use_cuda else ["CPUExecutionProvider"])

options = SessionOptions()

'''
if self.args.dynamic_quantize:
    model_path = quantize(Path(os.path.join(model_name, "onnx_model.onnx")))
    self.model = InferenceSession(
        model_path.as_posix(), options, providers=onnx_execution_provider
    )
else:
    model_path = os.path.join(model_name, "onnx_model.onnx")
    self.model = InferenceSession(
        model_path, options, providers=onnx_execution_provider
    )
'''

model_path = os.path.join('/content/drive/MyDrive/POSTAG/BERT_model/ONNXModel(Noquantize)/onnx_model.onnx') ### ไฟล์.onnx โมเดล ONNX ที่เรา train
model = InferenceSession(model_path, options, providers=onnx_execution_provider)

/usr/local/lib/python3.7/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:56: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  "Available providers: '{}'".format(name, ", ".join(available_provider_names)))


In [7]:
model

In [8]:
# ประกาศ tokenizer
config_class = CamembertConfig
model_class = CamembertForTokenClassification
tokenizer_class = CamembertTokenizer


model_name = '/content/drive/MyDrive/POSTAG/BERT_model/ONNXModel(Noquantize)'  # โฟลเดอร์ที่ เก็บโมเดลและ argument (เอาทั้งโฟลเดอร์)

tokenizer = tokenizer_class.from_pretrained(model_name, do_lower_case=False,)

In [9]:
tokenizer

PreTrainedTokenizer(name_or_path='/content/drive/MyDrive/POSTAG/BERT_model/ONNXModel(Noquantize)', vocab_size=25005, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<s>NOTUSED', '</s>NOTUSED', '<_>']})

In [10]:
class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(
        self,
        guid,
        words,
        labels,
        x0=None,
        y0=None,
        x1=None,
        y1=None,
        tokenized_word_ids=None,
    ):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
            x0: (Optional) list. The list of x0 coordinates for each word.
            y0: (Optional) list. The list of y0 coordinates for each word.
            x1: (Optional) list. The list of x1 coordinates for each word.
            y1: (Optional) list. The list of y1 coordinates for each word.
            tokenized_word_ids: (Optional) list. Tokenized words converted to input_ids
        """
        self.guid = guid
        self.words = words
        self.labels = labels
        self.tokenized_word_ids = tokenized_word_ids
        if x0 is None:
            self.bboxes = None
        else:
            self.bboxes = [[a, b, c, d] for a, b, c, d in zip(x0, y0, x1, y1)]

In [ ]:
def _convert_tokens_to_word_logits(
        input_ids, label_ids, attention_mask, logits
    ):

        ignore_ids = [
            tokenizer.convert_tokens_to_ids(tokenizer.pad_token),
            tokenizer.convert_tokens_to_ids(tokenizer.sep_token),
            tokenizer.convert_tokens_to_ids(tokenizer.cls_token),
        ]

        # Remove unuseful positions
        masked_ids = input_ids[(1 == attention_mask)]
        masked_labels = label_ids[(1 == attention_mask)]
        masked_logits = logits[(1 == attention_mask)]
        for id in ignore_ids:
            masked_labels = masked_labels[(id != masked_ids)]
            masked_logits = masked_logits[(id != masked_ids)]
            masked_ids = masked_ids[(id != masked_ids)]

        # Map to word logits
        word_logits = []
        tmp = []
        for n, lab in enumerate(masked_labels):
            if lab != pad_token_label_id:
                if n != 0:
                    word_logits.append(tmp)
                tmp = [list(masked_logits[n])]
            else:
                tmp.append(list(masked_logits[n]))
        word_logits.append(tmp)

        return word_logits

In [ ]:
# แปลง Input format

predict_examples = [
                    InputExample(
                        i, # Unique id for the example.
                        sentence.split(), # The words of the sequence. split  โดยช่องว่าง
                        [_POS_TAGS[0] for word in sentence.split()], # ไม่ได้ใช้ตอน predict
            
                    )
                    for i, sentence in enumerate(to_predict) ###### วนแต่ละ sentence ใน list
                ]
print(predict_examples)

In [9]:
# Encode Input
model_inputs = tokenizer.batch_encode_plus(
                to_predict,
                return_tensors="np", #pt
                padding=True,
                truncation=True,
                is_split_into_words=(False),
            )

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [15]:
model_inputs

{'input_ids': array([[    5,    10,  2840,   369,  7711,    10,  6803,  6345,    10,
          288,  2246,    10,   265,    10,   301,    10,   326,    10,
          573,    10, 22751,     6]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [14]:
# Change shape for batching
encoded_model_inputs = []
for (input_ids, attention_mask) in tqdm(zip(model_inputs["input_ids"], model_inputs["attention_mask"])):
    encoded_model_inputs.append((input_ids, attention_mask))
print(encoded_model_inputs)

0it [00:00, ?it/s]

[(array([    5,    10,  2840,   369,  7711,    10,  6803,  6345,    10,
         288,  2246,    10,   265,    10,   301,    10,   326,    10,
         573,    10, 22751,     6]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))]


In [15]:
# Set up Batches
eval_batch_size = 8 # ไม่ได้ใช้แต่ใส่ไม่ให้ error
silent = False
preds = None
eval_sampler = SequentialSampler(encoded_model_inputs)
eval_dataloader = DataLoader(
                encoded_model_inputs,
                sampler=eval_sampler,
                batch_size=eval_batch_size,)

             

In [ ]:
##### test ######
input_ids = model_inputs["input_ids"]
attention_mask = model_inputs["attention_mask"]
inputs_onnx = {
                          "input_ids": input_ids,
                          "attention_mask": attention_mask,
                      }

output = model.run(None, inputs_onnx)
print(output)

In [12]:
output = np.array(output)
output.shape

(1, 1, 22, 16)

In [11]:
for batch in tqdm(
                eval_dataloader, disable=silent, desc="Running Prediction"
            ):
    #print(batch[0].detach().cpu().numpy())
    #print(batch[1].detach().cpu().numpy())
    inputs_onnx = {
                          "input_ids": batch[0].detach().cpu().numpy(),
                          "attention_mask": batch[1].detach().cpu().numpy(),
                      }
    # Inference raw output
    output = model.run(None, inputs_onnx)
    print(output)
    if preds is None:
        preds = output[0]
        out_input_ids = inputs_onnx["input_ids"]
        out_attention_mask = inputs_onnx["attention_mask"]
    else:
        preds = np.append(preds, output[0], axis=0)
        out_input_ids = np.append(
            out_input_ids, inputs_onnx["input_ids"], axis=0
        )
        out_attention_mask = np.append(
            out_attention_mask, inputs_onnx["attention_mask"], axis=0
        )
  

NameError: ignored

In [17]:
preds.shape

(1, 22, 16)

In [18]:
pad_token_label_id = -100
out_label_ids = [[] for _ in range(len(to_predict))]
max_len = len(out_input_ids[0])

for index, sentence in enumerate(to_predict):
    for word in sentence.split():
        word_tokens = tokenizer.tokenize(word)
        out_label_ids[index].extend(
        [0] + [pad_token_label_id] * (len(word_tokens) - 1)
    )
    out_label_ids[index].insert(0,pad_token_label_id)
    out_label_ids[index].append(pad_token_label_id)

    if len(out_label_ids[index]) < max_len:
        out_label_ids[index].extend([-100] * (max_len-len(out_label_ids[index])))

out_label_ids = np.array(out_label_ids).reshape(len(out_label_ids), max_len)

In [19]:
# predict output
token_logits = preds
preds = np.argmax(preds, axis=2)

In [20]:
token_logits[0][0]

array([ 4.0018024,  1.3319618, -1.5673672,  4.7854166,  0.7431917,
       -0.4197538,  0.096866 , -2.3110754, -1.2437447, -2.5236938,
       -1.142465 , -1.6213455, -3.585608 , -1.7706078, -2.7699332,
       -3.86404  ], dtype=float32)

In [31]:
preds

array([[{'ไ่ก่': 'NN'}, {'จิก': 'VV'}, {'เด็ก': 'NN'}, {'ตาย': 'VV'},
        {'คน': 'CL'}, {'_': 'PU'}, {'เกิด': 'VV'}, {'บน': 'PS'},
        {'ปาก': 'NN'}, {'โอ่ง': 'NN'}]], dtype=object)

In [22]:
# map label
label_map = {i: label for i, label in enumerate(_POS_TAGS)}

In [ ]:
label_map

In [23]:
# map word แต่ละคำ กับ POS/NER ในรูปแแบบ dict
out_label_list = [[] for _ in range(out_label_ids.shape[0])]
preds_list = [[] for _ in range(out_label_ids.shape[0])]
for i in range(out_label_ids.shape[0]):
    for j in range(out_label_ids.shape[1]):
        if out_label_ids[i, j] != pad_token_label_id:
            out_label_list[i].append(label_map[out_label_ids[i][j]])
            preds_list[i].append(label_map[preds[i][j]])

preds = [
            [
              {word: preds_list[i][j]}
              for j, word in enumerate(sentence.split()[: len(preds_list[i])])
            ]
              for i, sentence in enumerate(to_predict)
        ]

In [24]:
preds

[[{'ไ่ก่': 'NN'},
  {'จิก': 'VV'},
  {'เด็ก': 'NN'},
  {'ตาย': 'VV'},
  {'คน': 'CL'},
  {'_': 'PU'},
  {'เกิด': 'VV'},
  {'บน': 'PS'},
  {'ปาก': 'NN'},
  {'โอ่ง': 'NN'}]]

In [25]:
# map word แต่ละคำ กับ logit array ในรูปแแบบ dict

word_tokens = []
for n, sentence in enumerate(to_predict):
    w_log = _convert_tokens_to_word_logits(
        out_input_ids[n],
        out_label_ids[n],
        out_attention_mask[n],
        token_logits[n],
    )
    word_tokens.append(w_log)

model_outputs = [
                [
                    {word: word_tokens[i][j]}
                    for j, word in enumerate(sentence.split()[: len(preds_list[i])])
                ]
                for i, sentence in enumerate(to_predict)
            ]

NameError: ignored

In [ ]:
model_outputs